# Neural Radiance Fields

### What Problem are we trying to solve?
#### View Synthesis

### what are Neural Randiance Fields?

- 3d neural network encoding for continous viewpoints
we want to over fit a neural network to a single point- 
- The weights of the Neural Networks represent the Scene


#### Positional Encoding


#### Represent a scene as a continous 5D function 

***Input:***
- $$(x,y,z,\theta, \phi)$$
    - $$(x,y,z) - Coordinates$$
    - $$(\theta, \phi) - viewing\;direction$$

***Output:***
- $$(r,g,b,\sigma)$$
    - $$(r,g,b) - color\;channels$$
    - $$\sigma - density$$


## Import Statements

In [4]:
import torch
from torch.utils.data import DataLoader
import numpy as np
from tqdm import tqdm
import torch.nn as nn
import matplotlib as plt


## Defining the Neural network structure

In [5]:
class Nerf(nn.Module):
    def __init__(self, pos_dim=10, dir_dim =4, hidden_dim = 128):
        super().__init__()
        
        self.dir_dim = dir_dim
        self.pos_dim = pos_dim
        
        
        self.b1 = nn.Sequential(nn.Linear(pos_dim*6+3, hidden_dim), nn.ReLU(),
                                nn.Linear(hidden_dim,hidden_dim), nn.ReLU(),
                                nn.Linear(hidden_dim,hidden_dim), nn.ReLU(),
                                nn.Linear(hidden_dim,hidden_dim), nn.ReLU()
                               )
        
        self.b2 = nn.Sequential(nn.Linear(pos_dim*6+3, hidden_dim), nn.ReLU(),
                                nn.Linear(hidden_dim,hidden_dim), nn.ReLU(),
                                nn.Linear(hidden_dim,hidden_dim), nn.ReLU(),
                                nn.Linear(hidden_dim, hidden_dim+1)
                               )
        
        self.b3 = nn.Sequential(nn.Linear(dir_dim*6+hidden_dim+3, hidden_dim//2), nn.ReLU()
                               )
        
        self.b4 = nn.Sequential(nn.Linear(hidden_dem//2, 3), nn.Sigmoid()
                               )
        
    @staticmethod
    def pos_encoding(x, L):
        out = [x]
        
        for i in range(L):
            out.append(torch.sin(2**i*x))
            out.append(torch.cos(2**i*x))
            
        return torch.cat(out, dim=1)
    
    def forward(self, o, d):
        emb_x = self.pos_encoding(o, self.pos_dim)
        emb_d = self.pos_encoding(d, self.dir_dim)
        
        h = self.b1(emb_x)
        tmp = self.b2(torch.cat((h,emb_x), dim=1))
        h,sigma = tmp[:,:-1], self.relu(tmp[:,-1])
        h = self.b3(toch.cat(h,emb_d), dim=1)
        c = self.b4(h)
        
        return x, simga
    

In [ ]:
def accumulated_transmittance(alphas):
    at = torch.cumprod(alphas,1)
    
    return torch.cat((torch.ones((at.shape[0],1), device=alphas.device), at[:,:-1]), dim=1)

def render_rays(nerf_model, ray_origins, ray_directions, hn=0, hf=0.5, nb_bins=192):
    